# AiBD Laboratorium 3 - Kacper Motyka

## Łączenie z bazą danych

In [2]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Zadanie 1. Ile kategorii filmów mamy w wypożyczalni?

In [3]:
number_of_categories = pd.read_sql('SELECT  DISTINCT category_id FROM category',con=connection).count()
print(number_of_categories)

category_id    16
dtype: int64


### Odpowiedź: W wypożyczalni mamy 16 kategorii filmów.

## Zadanie 2. Wyświetl listę kategorii w kolejności alfabetycznej.

In [4]:
categories_ordered = pd.read_sql('SELECT name FROM category ORDER BY name',con=connection)
print(categories_ordered)

           name
0        Action
1     Animation
2      Children
3      Classics
4        Comedy
5   Documentary
6         Drama
7        Family
8       Foreign
9         Games
10       Horror
11        Music
12          New
13       Sci-Fi
14       Sports
15       Travel


## Zadanie 3. Znajdź najstarszy i najmłodszy film do wypożyczenia.

In [5]:
oldest_newest__film = pd.read_sql('SELECT title FROM film ORDER BY release_year',con=connection)
print(oldest_newest__film)

                 title
0      Chamber Italian
1     Grosse Wonderful
2      Airport Pollock
3    Bright Encounters
4     Academy Dinosaur
..                 ...
995     Young Language
996         Youth Kick
997       Zhivago Core
998  Zoolander Fiction
999          Zorro Ark

[1000 rows x 1 columns]


### Odpowiedź: Posortowałem według dat wszystkie filmy, od najstarszego do najnowszego, stąd wiem, że najstarszy film to 'Chamber Italian' a najnowszy film to 'Zorro Ark'.

## Zadanie 4. Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?

In [6]:
between_film = pd.read_sql("SELECT COUNT(*) FROM rental WHERE rental_date BETWEEN '2005-07-01'::timestamp AND '2005-08-01'::timestamp",con=connection)
print(between_film)

   count
0   6709


### Odpowiedź: Pomiędzy 2005-07-01 a 2005-08-01 było 6709 wypożyczeń.

## Zadanie 5. Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?

In [7]:
between_film = pd.read_sql("SELECT COUNT(*) FROM rental WHERE rental_date BETWEEN '2010-01-01 00:00:00'::timestamp AND '2011-02-01 23:59:59'::timestamp",con=connection)
print(between_film)

   count
0      0


### Odpowiedź: Pomiędzy 2010-01-01 a 2011-02-01 było 0 wypożyczeń, ponieważ dane pochodzą ze wcześniejszych lat.

## Zadanie 6. Znajdź największą płatność wypożyczenia.

In [8]:
highest_payment = pd.read_sql("SELECT MAX(amount) FROM payment",con=connection)
print(highest_payment)

     max
0  11.99


### Odpowiedź: Największa płatność wynosiła 11.99.

## Zadanie 7. Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.

In [9]:
#WHERE country.country IN ('Poland', 'Nigeria', 'Bangladesh')"

pol_nig_ban = pd.read_sql("SELECT customer.first_name, customer. last_name, \
                           country.country \
                           FROM country \
                           LEFT JOIN city \
                           ON country.country_id = city.country_id \
                           LEFT JOIN address \
                           ON city.city_id = address.city_id \
                           LEFT JOIN customer \
                           ON address.address_id = customer.address_id \
                           WHERE country.country IN ('Poland', 'Nigeria', 'Bangladesh') \
                           ",
                           con=connection)
print(pol_nig_ban)

   first_name  last_name     country
0       Sonia    Gregory     Nigeria
1       Brian      Wyman      Poland
2      Sidney   Burleson      Poland
3      Rodney    Moeller     Nigeria
4     Stephen     Qualls  Bangladesh
5       Velma      Lucas     Nigeria
6     Marilyn       Ross     Nigeria
7       Elsie     Kelley     Nigeria
8      Gladys   Hamilton     Nigeria
9       Frank   Waggoner  Bangladesh
10   Marjorie     Tucker      Poland
11      Carol     Garcia     Nigeria
12       Leah     Curtis      Poland
13      Ruben      Geary      Poland
14       Olga    Jimenez     Nigeria
15     Bertha   Ferguson     Nigeria
16     Tracey    Barrett     Nigeria
17         Jo     Fowler     Nigeria
18    Johnnie   Chisholm      Poland
19    Wallace      Slone     Nigeria
20   Michelle      Clark  Bangladesh
21    Russell    Brinson      Poland
22     Jimmie  Eggleston      Poland
23  Constance       Reid     Nigeria


### Odpowiedź: Jest 24 takich klientów, wyżej dla przejrzystości wyświetliłem jedynie imię, nazwisko oraz kraj z którego klient pochodzi.

## Zadanie 8. Gdzie mieszkają członkowie personelu?


In [22]:
addresses_of_personnel = pd.read_sql("SELECT staff.first_name, staff.last_name, \
                           country, city, address.address \
                           FROM staff \
                           LEFT JOIN address \
                           ON staff.address_id = address.address_id \
                           LEFT JOIN city \
                            ON address.city_id = city.city_id \
                            LEFT JOIN country \
                            ON city.country_id = country.country_id", 
                           con=connection)
print(addresses_of_personnel)

  first_name last_name    country        city               address
0       Mike   Hillyer     Canada  Lethbridge     23 Workhaven Lane
1        Jon  Stephens  Australia   Woodridge  1411 Lillydale Drive


### Odpowiedź: W wypożyczalni pracują dwie osoby o następujących adresach. Aby uzyskać informacje o adresach należało dołączyć informacje z datasetu address za pomocą LEFT JOIN.

## Zadanie 9. Ilu pracowników mieszka w Argentynie lub Hiszpanii?

In [11]:
countries_of_personnel = pd.read_sql("SELECT staff.first_name, staff. last_name, \
                                      country.country \
                                      FROM staff \
                                      LEFT JOIN address \
                                      ON staff.address_id = address.address_id \
                                      LEFT JOIN city \
                                      ON address.city_id = city.city_id \
                                      LEFT JOIN country \
                                      ON city.country_id = country.country_id",
                                      con=connection)

count_of_personnel = pd.read_sql("SELECT COUNT(*) \
                                  FROM staff \
                                  LEFT JOIN address \
                                  ON staff.address_id = address.address_id \
                                  LEFT JOIN city \
                                  ON address.city_id = city.city_id \
                                  LEFT JOIN country \
                                  ON city.country_id = country.country_id \
                                  WHERE country.country IN ('Argentina', 'Spain')",
                                  con=connection)

print(countries_of_personnel)
print("\nNumber of personel from Argentina or Spain: \n", count_of_personnel)

  first_name last_name    country
0       Mike   Hillyer     Canada
1        Jon  Stephens  Australia

Number of personel from Argentina or Spain: 
    count
0      0


### Odpowiedź: Widać, że nikt z personelu nie pochodzi z Argentyny ani Hiszpani. Personel pochodzi z Kanady oraz Australii.

## Zadanie 10. Jakie kategorie filmów zostały wypożyczone przez klientów?

In [12]:
categories_of_rented = pd.read_sql("SELECT DISTINCT  category.name, count(category.name) \
                                    FROM rental \
                                    LEFT JOIN inventory \
                                    ON rental.inventory_id = inventory.inventory_id \
                                    LEFT JOIN film \
                                    ON inventory.film_id = film.film_id \
                                    LEFT JOIN film_category \
                                    ON film.film_id = film_category.film_id \
                                    LEFT JOIN category \
                                    ON film_category.category_id = category.category_id \
                                    GROUP BY category.name",
                                    con=connection)
print(categories_of_rented)

           name  count
0        Action   1112
1     Animation   1166
2      Children    945
3      Classics    939
4        Comedy    941
5   Documentary   1050
6         Drama   1060
7        Family   1096
8       Foreign   1033
9         Games    969
10       Horror    846
11        Music    830
12          New    940
13       Sci-Fi   1101
14       Sports   1179
15       Travel    837


### Odpowiedź: Generalnie kolejnymi LEFT JOINami przechodzę po drzewku przedstawiającym schemat bazy danych do tej informacji, któa mnie interesuje. Jak widać wszystkie kategorie filmów zostały wypożyczone.

## Zadanie 11. Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.

In [26]:
categories_of_rented_in_america = pd.read_sql("SELECT DISTINCT  category.name, count(category.name) \
                                              FROM rental \
                                              LEFT JOIN inventory \
                                              ON rental.inventory_id = inventory.inventory_id \
                                              LEFT JOIN film \
                                              ON inventory.film_id = film.film_id \
                                              LEFT JOIN film_category \
                                              ON film.film_id = film_category.film_id \
                                              LEFT JOIN category \
                                              ON film_category.category_id = category.category_id \
                                              LEFT JOIN customer \
                                              ON customer.customer_id = rental.customer_id \
                                              LEFT JOIN address \
                                              ON address.address_id = customer.address_id \
                                              LEFT JOIN city \
                                              ON address.city_id = city.city_id \
                                              LEFT JOIN country \
                                              ON country.country_id = city.country_id \
                                              WHERE country.country IN ('United States') \
                                              GROUP BY category.name \
                                              ",
                                              con=connection)
print(categories_of_rented_in_america)

           name  count
0        Action     62
1     Animation     68
2      Children     56
3      Classics     54
4        Comedy     59
5   Documentary     74
6         Drama     69
7        Family     66
8       Foreign     64
9         Games     60
10       Horror     62
11        Music     51
12          New     55
13       Sci-Fi     62
14       Sports     73
15       Travel     33


### Odpowiedź: W Ameryce wypożoczono 968 filmów o przeróżnych kategoriach.

## Zadanie 12. Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley.

In [15]:
OP_or_JZ = pd.read_sql("SELECT film.title, actor.first_name, actor.last_name \
                        FROM film \
                        LEFT JOIN film_actor \
                        ON film.film_id = film_actor.film_id \
                        LEFT JOIN actor \
                        ON film_actor.actor_id = actor.actor_id \
                        WHERE (actor.first_name = 'Olympia' and actor.last_name = 'Pfeiffer') or \
                              (actor.first_name = 'Julia' and actor.last_name = 'Zellweger') or \
                              (actor.first_name = 'Ellen' and actor.last_name = 'Presley') \
                        ", con=connection)

print(OP_or_JZ)

                title first_name  last_name
0     Bilko Anonymous      Ellen    Presley
1   Caribbean Liberty      Ellen    Presley
2    Casper Dragonfly      Ellen    Presley
3    Empire Malkovich      Ellen    Presley
4       Floats Garden      Ellen    Presley
..                ...        ...        ...
64       Outlaw Hanky      Julia  Zellweger
65    Panky Submarine      Julia  Zellweger
66   Rider Caddyshack      Julia  Zellweger
67          Won Dares      Julia  Zellweger
68      Wyoming Storm      Julia  Zellweger

[69 rows x 3 columns]


### Odpowiedź: Znaleziono 69 filmów w których grały powyższe aktorki.

## Wnioski

### Za pomocą SQL możemy w prosty sposob otrzymywać z bazy danych odpowiedzi na bardzo konkrentne pytania, dlatego też jest to bardzo użyteczne narzędzie.

### Możemy używać różnych funkcji, dzięki czemu możemy poznać na przykład ilość wystąpień danego zapytania.

### Za pomocą pythona i jego modułów takich jak pandas, psycopg2, sqlalchemy  możemy na kilka sposobów łączyć się z bazą danych.

### Dzięki tym bibliotekom możemy też używać języka SQL w pythonie co sprawia SQL'a jeszcze bardziej wszechstronnym.

### Mamy dostępnych wiele klauzul takich jak DISTINCT (który wybiera tylko jedną daną wartość zamiast na przykład dziesięciu), WHERE, za pomocą której możemy przeszukiwać bazę danych z zadanymi warunkami i wiele więcej.

### Powyższe ćwiczenie nauczyło nas używania podstawowych QUERY, dzięki czemu zdobyliśmy przydatną wiedzę, którą będziemy mogli wykorzystać w przyszłości.